In [10]:
import pandas as pd
import numpy as np
import math

import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [17]:
df = pd.read_csv("outputs/scores2.csv")
df

,Unnamed: 0,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,...,fscore,dist2opt,tp,tn,fp,fn,nodestates,arcs,items/nodestates,items/arcs
0,0,1000,4,1,0.999999,1,2,3OFF2,AIC,0.666667,...,0.800000,0.333333,2,9,0,1,16,3,62.500000,333.333333
1,1,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,...,0.000000,1.414214,0,7,2,3,16,3,62.500000,333.333333
2,2,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,...,0.000000,1.414214,0,7,2,3,16,3,62.500000,333.333333
3,3,1000,4,1,0.999999,1,2,3OFF2,AIC,0.333333,...,0.400000,0.833333,1,8,1,2,16,3,62.500000,333.333333
4,4,1000,4,1,0.999999,1,2,3OFF2,AIC,0.000000,...,0.000000,1.414214,0,7,2,3,16,3,62.500000,333.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,129785,10000,8,3,2.000000,1,4,TABU,K2,0.527273,...,0.495726,0.711878,29,464,33,26,65536,16,0.152588,625.000000
129786,129786,10000,8,3,2.000000,1,4,TABU,K2,0.672727,...,0.587302,0.580023,37,463,34,18,65536,16,0.152588,625.000000
129787,129787,10000,8,3,2.000000,1,4,TABU,K2,0.890909,...,0.830508,0.247555,49,483,14,6,65536,16,0.152588,625.000000
129788,129788,10000,8,3,2.000000,1,4,TABU,K2,1.000000,...,0.880000,0.214286,55,482,15,0,65536,16,0.152588,625.000000


In [18]:
df1 = pd.get_dummies(df, columns=["algorithm","scorefunction"], dtype=int)
df1

,Unnamed: 0,items,nodes,timesteps,arcs_ratio,islands,values,recall,precision,fscore,...,algorithm_3OFF2,algorithm_HC,algorithm_MIIC,algorithm_MMHC,algorithm_TABU,scorefunction_AIC,scorefunction_BDEU,scorefunction_BIC,scorefunction_K2,scorefunction_L2L
0,0,1000,4,1,0.999999,1,2,0.666667,1.000000,0.800000,...,1,0,0,0,0,1,0,0,0,0
1,1,1000,4,1,0.999999,1,2,0.000000,0.000000,0.000000,...,1,0,0,0,0,1,0,0,0,0
2,2,1000,4,1,0.999999,1,2,0.000000,0.000000,0.000000,...,1,0,0,0,0,1,0,0,0,0
3,3,1000,4,1,0.999999,1,2,0.333333,0.500000,0.400000,...,1,0,0,0,0,1,0,0,0,0
4,4,1000,4,1,0.999999,1,2,0.000000,0.000000,0.000000,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,129785,10000,8,3,2.000000,1,4,0.527273,0.467742,0.495726,...,0,0,0,0,1,0,0,0,1,0
129786,129786,10000,8,3,2.000000,1,4,0.672727,0.521127,0.587302,...,0,0,0,0,1,0,0,0,1,0
129787,129787,10000,8,3,2.000000,1,4,0.890909,0.777778,0.830508,...,0,0,0,0,1,0,0,0,1,0
129788,129788,10000,8,3,2.000000,1,4,1.000000,0.785714,0.880000,...,0,0,0,0,1,0,0,0,1,0


In [19]:
%%time
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    combinations = list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1,len(s)+1)))
    
    merged = []
    for combination in combinations:
        merged.append(np.concatenate(combination, axis=None))
    return merged
    
item_c = ["items"]
nodes_c = ["nodes"]
arc_c = ["arcs_ratio"]
times_c = ["timesteps"]
values_c = ["values"]
algorithm_c = ["algorithm_3OFF2","algorithm_HC","algorithm_MIIC","algorithm_MMHC","algorithm_TABU"]
score_c = ["scorefunction_AIC","scorefunction_BDEU","scorefunction_BIC","scorefunction_K2","scorefunction_L2L"]
nodestates = ["nodestates"]
arcs = ["arcs"]
items_nodestates = ["items/nodestates"]
items_arcs = ["items/arcs"]


columns_c = [item_c, nodes_c, arc_c, times_c, values_c, algorithm_c, score_c, nodestates, arcs, items_nodestates, items_arcs]

column_sets = powerset(columns_c)

results = pd.DataFrame()

columns = column_sets[-1]

regr = LinearRegression()


for col in df1.columns:
    df1[col] = np.random.permutation(df1[col])
    
    
regr.fit(df1[columns], df1["fscore"])

results =  pd.concat([results, pd.DataFrame({"items":["items" in columns], "nodes":["nodes" in columns], "arcs_ratio":["arcs_ratio" in columns], "timesteps":["timesteps" in columns], "values":["values" in columns], "algorithm":["algorithm_HC" in columns], "scorefunction":["scorefunction_AIC" in columns], "nodestates":["nodestates" in columns], "arcs":["arcs" in columns], "items/nodestates":["items/nodestates" in columns], "items/arcs":["items/arcs" in columns], "score":[regr.score(df1[columns], df1["fscore"])]})], ignore_index=True)

results

CPU times: total: 266 ms
Wall time: 541 ms


,items,nodes,arcs_ratio,timesteps,values,algorithm,scorefunction,nodestates,arcs,items/nodestates,items/arcs,score
0,True,True,True,True,True,True,True,True,True,True,True,0.000093
